# 10. GraphRAG for Stock Exchange ITOps: Unlocking Relationships Between Services and Incidents

## Introduction

Welcome to the 10th notebook in our series on **AI for Stock Exchange IT Operations**! In this notebook, we explore the revolutionary potential of **Graph-based Retrieval-Augmented Generation (GraphRAG)** and demonstrate how it can uncover relationships in complex IT environments.

By leveraging **GraphRAG**, IT operators can gain deep insights into service dependencies, configuration impacts, and historical incidents. This approach enhances traditional RAG techniques by incorporating graph structures to better represent interconnected systems, enabling precise and contextual retrieval of operational data.

### Objectives

By the end of this notebook, you will:

1. Understand the foundational principles of **GraphRAG** and how it differs from standard RAG.
2. Explore how graphs model relationships in IT Operations (e.g., service dependencies, incident histories).
3. Construct a graph-based knowledge base using service and incident data.
4. Query the graph to retrieve actionable insights for ITOps tasks.
5. Combine graph queries with LLMs to generate human-readable explanations and recommendations.

### What Is GraphRAG?

**GraphRAG** is an advanced variant of Retrieval-Augmented Generation (RAG) that incorporates **graph-based knowledge retrieval** into the workflow. Unlike traditional RAG, which relies on unstructured document retrieval, GraphRAG uses graph databases to model and query complex relationships between entities.

#### Why GraphRAG?

- **Relational Insights**: Graphs allow you to explicitly represent dependencies, hierarchies, and connections between entities like services, configurations, and incidents.
- **Precision**: Queries target specific nodes and relationships, reducing ambiguity and noise in retrieval results.
- **ITOps Relevance**: Perfect for environments with interconnected systems, such as stock exchanges, where understanding dependencies is critical.

### How GraphRAG Works

#### Step 1: Graph Construction
- Use a graph database (e.g., Neo4j) to store relationships between services, incidents, and configurations.
- Populate the graph with nodes (entities) and edges (relationships) extracted from IT operational data.

#### Step 2: Graph Querying
- Use graph queries (e.g., Cypher) to retrieve precise contextual data, such as impacted services or related incidents.

#### Step 3: Integration with LLM
- Feed the retrieved graph context into an LLM.
- Generate clear and actionable insights, such as explanations of root causes or recommendations for configuration changes.

### First Interaction with GraphRAG

Let’s dive into how GraphRAG works by building a simple knowledge graph for our stock exchange environment. In this example, we will:
1. Construct a graph from service and incident data.
2. Query the graph to retrieve relationships and dependencies.
3. Use an LLM to provide explanations and actionable recommendations.

![image](images/graph.png)

## 1. Installing the Required Libraries

Before we start, we need to install the necessary libraries.

In [18]:
%pip install langchain langchain-community langchain-ollama langchain-experimental neo4j langchain_core --quiet 

Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import Driver
from pydantic import BaseModel, Field
from neo4j import GraphDatabase
import os

## 2. Initialize the Neo4j Graph Database

### Set Up Connection

We first connect to the Neo4j database to prepare for storing and querying graph data.

In [20]:
def initialize_graph_connection(uri: str, username: str, password: str):
    """Initializes connection to Neo4j graph database."""
    try:
        graph = Neo4jGraph(url=uri, username=username, password=password)
        print("Connected to Neo4j successfully!")
        return graph
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")
        raise


# Connection Details
URI = "neo4j://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "neo4jneo4j"

# Initialize Graph
graph = initialize_graph_connection(URI, USERNAME, PASSWORD)

Connected to Neo4j successfully!


## 3. Load and Split Text Data

### Load Documents
Load text data from a file to create graph nodes and relationships.

In [21]:
# Load data from file
loader = TextLoader(file_path="graph/post-mortem.txt")
docs = loader.load()
print(f"Loaded {len(docs)} documents.")

Loaded 1 documents.


### Split Documents

In real-world IT environments, operational data and incident reports are often lengthy and unstructured. To make this information usable for both graph-based analysis and AI processing:

- **Improved Context Handling**: Splitting large documents ensures that each chunk contains focused and manageable information, making it easier for LLMs to understand and process context effectively.
- **Efficient Graph Population**: Smaller chunks allow us to represent detailed relationships in the graph without overwhelming the structure with overly broad or ambiguous data.
- **Scalability**: Processing smaller chunks reduces computational overhead and avoids issues with memory constraints or truncation limits of AI models.

Below, we split the documents into smaller pieces, ensuring meaningful overlap between chunks to preserve context.

In [22]:
# Split text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)
print(f"Split documents into {len(documents)} chunks.")

Split documents into 15 chunks.


## 4. Convert Documents into Graph-Compatible Data

Once we have split the documents into smaller, manageable chunks, the next step is to extract entities and relationships from the text. This is essential for building a graph that represents dependencies and connections in our stock exchange IT operations environment.

### Why Use an LLM Transformer?

- **Entity Extraction**: Identifies key entities such as services, incidents, and teams from the text.
- **Relationship Discovery**: Maps connections between entities, providing a foundation for the graph.
- **Contextual Graph Representation**: Converts unstructured text into structured, graph-compatible data.

Below, we use the `LLMGraphTransformer` with the `OllamaFunctions` LLM to process the text chunks and generate graph documents.

In [23]:
llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [24]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Itops Team', type='Team', properties={}), Node(id='Stock Exchange', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Itops Team', type='Team', properties={}), target=Node(id='Stock Exchange', type='Organization', properties={}), type='INCIDENT_RESPONSE', properties={})], source=Document(metadata={'source': 'graph/post-mortem.txt'}, page_content='### Post-Mortem Report for ItOps Team: Stock Exchange Incident'))

### Adding Graph Documents to the Graph Database

After converting the text documents into graph-compatible data, the next step is to populate the graph database with this information.

#### Why Populate the Graph?
- **Graph Representation**: The documents are transformed into nodes and relationships, creating a graph structure that models dependencies and interactions in the stock exchange IT environment.
- **Query Capability**: Adding these documents allows us to query the graph for insights into service impacts, incidents, and more.
- **Source Traceability**: By including the source text, we can trace each node and relationship back to its origin for deeper analysis.

Below, we add the converted documents to the Neo4j graph, setting `baseEntityLabel` to create base nodes and `include_source` to retain the original context.

In [25]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)

## 5. Creating Embeddings and Vector Index for the Graph

To enable efficient similarity-based retrieval of graph nodes, we generate embeddings for the text data and create a vector index. This allows us to:

- **Enhance Search**: Retrieve relevant nodes based on vector similarity, useful for finding related incidents or services.
- **Hybrid Search**: Combine traditional graph queries with vector-based retrieval for more robust results.
- **AI-Driven Contextual Queries**: Use embeddings to find semantically similar nodes, enhancing the capability of GraphRAG.

Below, we:
1. Use the `OllamaEmbeddings` model to generate embeddings for the graph data.
2. Create a vector index in Neo4j using the `Neo4jVector` module.
3. Configure a retriever for querying the vector index.

In [26]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)
vector_retriever = vector_index.as_retriever()

## 6. Creating a Full-Text Index in the Graph Database

To enable efficient text-based searches, we create a **full-text index** in the Neo4j database. This allows for:

- **Improved Search Performance**: Quickly retrieve nodes based on their properties.
- **Enhanced Query Capabilities**: Use full-text search to find entities or relationships by their descriptions or identifiers.

Below, we define a Cypher query to create a full-text index on the `id` property of nodes labeled `__Entity__`. This will help us efficiently search for entities in the graph.

In [27]:
# Authentication credentials
AUTH = (USERNAME, PASSWORD)

# Establish a connection to the Neo4j driver
driver = GraphDatabase.driver(
    uri=URI,
    auth=AUTH,
)


# Function to create a full-text index
def create_fulltext_index(tx):
    query = """
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    """
    tx.run(query)


# Wrapper function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Full-text index created successfully.")


# Attempt to create the full-text index
try:
    create_index()
except Exception as e:
    print(f"Error creating full-text index: {e}")

# Close the Neo4j driver connection
driver.close()

Error creating full-text index: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=7, name='fulltext_entity_id', type='FULLTEXT', schema=(:__Entity__ {id}), indexProvider='fulltext-1.0' )'.}


## 7. Defining the Entity Extraction Class and Prompt

To extract key entities (such as incidents, systems, teams, and protocols) from the text, we define:

1. **`Entities` Class**: A Pydantic model to specify the schema of the extracted entities.
2. **Prompt Template**: A structured prompt for the LLM, instructing it to focus on extracting relevant entities for stock exchange incident management.

This approach ensures that the LLM produces well-structured outputs, aligning with the graph schema for seamless integration.

Below, we set up the `Entities` class and the `ChatPromptTemplate`.

In [28]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the relevant entities, such as incidents, systems, teams, or protocols, "
        "that appear in the text.",
    )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Your task is to extract key entities related to stock exchange incident management. "
            "Focus on identifying incidents, systems, teams, protocols, and any related actions or impacts "
            "from the provided text. Ensure the extracted entities align with the database schema "
            "and are relevant to troubleshooting and resolution.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [29]:
entity_chain.invoke(
    "Who's the Infrastructure Team?"
)

Entities(names=['Infrastructure Team'])

## 9. Retrieving Graph Data Based on Full-Text Index and Entities

The `graph_retriever` function performs the following tasks:

1. **Entity Extraction**: It uses the `entity_chain` to extract relevant entities from the input question.
2. **Full-Text Search**: Queries the Neo4j full-text index (`fulltext_entity_id`) to locate nodes related to the extracted entities.
3. **Neighborhood Exploration**: Collects the relationships (edges) between the retrieved nodes and their neighbors.

This approach ensures that the retrieved data is relevant to the user's query, focusing on the entities and their connections in the graph.

Below is the implementation of the `graph_retriever` function.

In [30]:
# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el["output"] for el in response])
    return result

In [31]:
print(graph_retriever("Who's the Infrastructure Team?"))

Itops Team - INCIDENT_RESPONSE -> Stock Exchange
Team Training On Incident Response - ASSIGNED_TO -> Training Coordinator


### Combining Graph and Vector Retrieval for Enhanced Context

The `full_retriever` function combines:

1. **Graph Retrieval**: Uses the `graph_retriever` function to extract relationships and entities from the graph database.
2. **Vector Retrieval**: Leverages the vector retriever to find semantically similar documents based on the input question.

By merging these two approaches, this function provides comprehensive and contextual information that can be used by an LLM for generating insights and recommendations.

#### Output Structure
The combined output includes:
- **Graph Data**: Relationships and entities retrieved from the graph.
- **Vector Data**: Semantically similar document content retrieved using embeddings.

Below is the implementation.

In [32]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

## 11. Creating and Executing the Query Chain

The query chain integrates all the components into a seamless pipeline for answering questions:

1. **Context Generation**: Combines graph-based and vector-based retrieval using the `full_retriever` function to provide rich contextual information.
2. **Prompt Template**: A predefined template instructs the LLM to focus on the given context and provide concise answers.
3. **Execution Chain**: The pipeline connects the context generator, prompt template, and LLM to process queries end-to-end.

### Key Features
- **Natural Language Responses**: The output is optimized for readability and clarity.
- **Conciseness**: The LLM is instructed to generate brief yet informative answers.
- **Integration**: Combines structured graph data with unstructured vector-based retrieval for robust insights.

Below is the implementation of the chain and an example query execution.

In [33]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": full_retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
chain.invoke(
    input="What the Infrastructure Team did?"
)

'The Infrastructure Team fixed database failover configuration, promoted a secondary node to restore operations, cleared transaction backlogs without inconsistencies, and communicated updates to stakeholders.'

## 12. Conclusion

In this notebook, we explored how to use **GraphRAG** to enhance IT operations for stock exchanges by combining graph-based knowledge retrieval and LLMs. This approach provided a robust framework for uncovering relationships between services, incidents, and dependencies in complex IT environments.

### Key Highlights
1. **Graph-Based Context Retrieval**:
   - By constructing and querying a graph database, we modeled dependencies and interactions between services, configurations, and incidents, enabling precise and context-aware retrieval of data.
2. **Seamless Integration with LLMs**:
   - We combined graph retrieval with vector-based document search to provide a rich, comprehensive context, allowing LLMs to generate concise and actionable insights.
3. **Enhanced Operational Insights**:
   - Through the pipeline, we demonstrated how IT operators can ask natural language questions and receive clear explanations and recommendations to address system incidents.

### What's Next?
In the next notebook, we will:
1. **Expand the GraphRAG Workflow**:
   - Introduce more advanced query capabilities and visualization tools for exploring the graph structure and relationships.
2. **Focus on Incident Automation**:
   - Use AI agents to proactively trigger incident remediation actions based on the insights retrieved from the graph.
3. **Integrate Advanced Models**:
   - Experiment with transformer-based models to enhance the reasoning and contextual understanding of operational data, further improving the accuracy and relevance of the recommendations.